<a href="https://colab.research.google.com/github/Alepescinaa/ScientificTools/blob/main/Project1/Cp3/tentativo_disperato_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import cvxpy as cvx
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import scipy.io
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import make_axes_locatable

!pip -q install pyDOE
from pyDOE import lhs

tf.random.set_seed(42)
np.random.seed(42)

  Preparing metadata (setup.py) ... done


In [3]:
!git clone https://github.com/Alepescinaa/ScientificTools
%cd ScientificTools/Project1/Cp3

Cloning into 'ScientificTools'...
remote: Enumerating objects: 657, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (55/55), done.
remote: Total 657 (delta 25), reused 1 (delta 1), pack-reused 601
Receiving objects: 100% (657/657), 140.38 MiB | 22.85 MiB/s, done.
Resolving deltas: 100% (223/223), done.
/content/ScientificTools/Project1/Cp3


In [4]:
CP3data = np.load("CP3data.npz")
CP3data = CP3data['arr_0']

CP3estimate = np.load("CP3field.npz")
CP3estimate = CP3estimate['arr_0']

In [7]:
mesh_size = 151**2
num_samples = 100
A = np.zeros((mesh_size, num_samples))

for i in range(num_samples):
  A[:, i] = CP3estimate[i][0].flatten()

[U,s,V] = np.linalg.svd(A, full_matrices=False)

k = 10
p = 100
perm = np.random.choice(mesh_size, size=p, replace=False)

C = np.zeros((p,mesh_size))
for i in range(p):
    C[i,perm[i]] = 1.0

Theta = C @ U
coeff_mat = np.zeros((k, 100))

for i in range (100):
  u = A[:,i]
  y = C @ u
  s_c = cvx.Variable(U.shape[1])
  constraints = [Theta @ s_c == y]
  obj = cvx.Minimize(cvx.norm(s_c, 1))
  prob = cvx.Problem(obj, constraints)
  prob.solve()
  coeff_mat[:, i] = s_c.value[:k]
  c1_min, c1_max = np.min(coeff_mat[0, :]), np.max(coeff_mat[0, :])
c2_min, c2_max = np.min(coeff_mat[1, :]), np.max(coeff_mat[1, :])
c3_min, c3_max = np.min(coeff_mat[2, :]), np.max(coeff_mat[2, :])
c4_min, c4_max = np.min(coeff_mat[3, :]), np.max(coeff_mat[3, :])
c5_min, c5_max = np.min(coeff_mat[4, :]), np.max(coeff_mat[4, :])
c6_min, c6_max = np.min(coeff_mat[5, :]), np.max(coeff_mat[5, :])
c7_min, c7_max = np.min(coeff_mat[6, :]), np.max(coeff_mat[6, :])
c8_min, c8_max = np.min(coeff_mat[7, :]), np.max(coeff_mat[7, :])
c9_min, c9_max = np.min(coeff_mat[8, :]), np.max(coeff_mat[8, :])
c10_min, c10_max = np.min(coeff_mat[9, :]), np.max(coeff_mat[9, :])

delta_c1 = c1_max - c1_min
delta_c2 = c2_max - c2_min
delta_c3 = c3_max - c3_min
delta_c4 = c4_max - c4_min
delta_c5 = c5_max - c5_min
delta_c6 = c6_max - c6_min
delta_c7 = c7_max - c7_min
delta_c8 = c8_max - c8_min
delta_c9 = c9_max - c9_min
delta_c10 = c10_max - c10_min

print(f"c1 in ({c1_min}, {c1_max})")
print(f"c2 in ({c2_min}, {c2_max})")
print(f"c3 in ({c3_min}, {c3_max})")
print(f"c4 in ({c4_min}, {c4_max})")
print(f"c5 in ({c5_min}, {c5_max})")
print(f"c6 in ({c6_min}, {c6_max})")
print(f"c7 in ({c7_min}, {c7_max})")
print(f"c8 in ({c8_min}, {c8_max})")
print(f"c9 in ({c9_min}, {c9_max})")
print(f"c10 in ({c10_min}, {c10_max})")

basis = U[:, :k]

c1 in (-14901.762094944293, -12144.026767270929)
c2 in (-1643.3543282374405, 1582.2151986049146)
c3 in (-1666.6222565530495, 1707.6163608834004)
c4 in (-776.3212422595338, 1280.5858314852646)
c5 in (-1093.9752515213913, 1047.6008504640922)
c6 in (-717.0424992189124, 537.6185969065431)
c7 in (-469.994381164961, 438.818028217973)
c8 in (-328.78631161812, 528.7552626478324)
c9 in (-257.8055527291008, 317.60595987413234)
c10 in (-216.03993095124412, 357.62370002461455)


In [8]:
# collocation points
Ncl = 151**2
Xcl = lhs(2,Ncl)
xcl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,0],dtype=tf.float64),axis=-1)
ycl = tf.expand_dims(tf.cast(-1.5+(3.0)*Xcl[:,1],dtype=tf.float64),axis=-1)
X_coll = tf.concat([xcl,ycl],1)

In [23]:
def penalty(param, lower_bound, upper_bound):
    return tf.reduce_sum(tf.square(tf.maximum(param - upper_bound, 0)) +
                         tf.square(tf.maximum(lower_bound - param, 0)))

# PINN loss function
def loss(xcl,ycl,xmeas,ymeas,umeas,coeff,regularization_strength):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = build_pinn(regularization_strength)(input_data)
    r_pred   = r_PINN(xcl,ycl,coeff,regularization_strength)

    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    y0 = tf.constant([-0.517409965],dtype=tf.float64)
    mse_bc= tf.pow( build_pinn(regularization_strength)( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), y0] ))  ) ,2)

    #penalty
    mse_penalty = penalty(coeff[0],0,1)+penalty(coeff[1],0,1)+penalty(coeff[2],0,1)+penalty(coeff[3],0,1)+penalty(coeff[4],0,1)+penalty(coeff[5],0,1)+penalty(coeff[6],0,1)+penalty(coeff[7],0,1)+penalty(coeff[8],0,1)+penalty(coeff[9],0,1)

    return mse_meas + mse_r + mse_bc + mse_penalty

def loss2(xcl,ycl,xmeas,ymeas,umeas,regularization_strength):
    input_data=tf.concat([xmeas,ymeas],1)
    umeas_pred = build_pinn(regularization_strength)(input_data)
    r_pred   = r_PINN2(xcl,ycl,regularization_strength)


    # loss components
    mse_meas  = tf.reduce_mean(tf.pow(umeas-umeas_pred,2))
    mse_r  = tf.reduce_mean(tf.abs(r_pred))

    # bc
    y0 = tf.constant([-0.517409965],dtype=tf.float64)
    mse_bc= tf.pow( build_pinn(regularization_strength)( tf.transpose( tf.stack( [tf.constant([1.5],dtype=tf.float64), y0] ))  ) ,2)

    return mse_meas + mse_r + mse_bc


@tf.function
def r_PINN(x,y,coeff,regularization_strength):
    input_data=tf.concat([x,y],1)
    u = build_pinn(regularization_strength)(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    theta_fiber = tf.constant([0.13757033666666668] ,dtype=tf.float64)
    a_ratio = tf.constant([5.896298503333333], dtype=tf.float64)
    theta0 = pi/2 - theta_fiber

    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / a_ratio * a[0]**2 + b[0]**2
    D_01 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / a_ratio * a[1]**2 + b[1]**2

    c1 = c1_min + delta_c1 * coeff[0]
    c2 = c2_min + delta_c2 * coeff[1]
    c3 = c3_min + delta_c3 * coeff[2]
    c4 = c4_min + delta_c4 * coeff[3]
    c5 = c5_min + delta_c5 * coeff[4]
    c6 = c6_min + delta_c6 * coeff[5]
    c7 = c7_min + delta_c7 * coeff[6]
    c8 = c8_min + delta_c8 * coeff[7]
    c9 = c9_min + delta_c9 * coeff[8]
    c10 = c10_min + delta_c10 * coeff[9]

    coeff_true = tf.expand_dims(tf.concat([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10], 0), 1)

    return   (((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y)))  - (1/(basis@coeff_true))**2

@tf.function
def r_PINN2(x,y):
    input_data=tf.concat([x,y],1)
    u = build_pinn(regularization_strength)(input_data)
    u_x = tf.gradients(u,x)[0]
    u_y = tf.gradients(u,y)[0]
    u_grad = tf.transpose(tf.concat([u_x, u_y], axis=1))

    pi = tf.constant(np.pi,dtype=tf.float64)
    theta_fiber = tf.constant([0.13757033666666668] ,dtype=tf.float64)
    a_ratio = tf.constant([5.896298503333333], dtype=tf.float64)
    theta0 = pi/2 - theta_fiber

    a = tf.stack([tf.cos(theta0), tf.sin(theta0)])
    b = tf.stack([tf.cos(theta0-pi/2), tf.sin(theta0-pi/2)])

    D_00 = 1 / a_ratio * a[0]**2 + b[0]**2
    D_01 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_10 = 1 / a_ratio * a[0] * a[1] + b[0] * b[1]
    D_11 = 1 / a_ratio * a[1]**2 + b[1]**2

    return  (((u_x * D_00 * u_x + u_x * D_01 * u_y + u_y * D_10 * u_x + u_y * D_11 * u_y)))  - (1/100)**2


# neural network weight gradients
@tf.function
def grad(model,xcl,ycl,xmeas,ymeas,umeas,coeff,regularization_strength):
    with tf.GradientTape(persistent=True) as tape:
        loss_value = loss(xcl,ycl,xmeas,ymeas,umeas,coeff,regularization_strength)
        grads = tape.gradient(loss_value,model.trainable_variables)
        grads_weight= tape.gradient(loss_value,coeff)
    return loss_value, grads, grads_weight
    # neural network weight gradients

@tf.function
def grad2(model,xcl,ycl,xmeas,ymeas,umeas,regularization_strength):
    with tf.GradientTape(persistent=True) as tape:
        loss_value2 = loss2(xcl,ycl,xmeas,ymeas,umeas,regularization_strength)
        grads2 = tape.gradient(loss_value2,model.trainable_variables)
    return loss_value2, grads2

In [19]:
from tensorflow.keras import regularizers

def build_pinn(data,regularization_strength):
    PINN = tf.keras.Sequential([
        tf.keras.layers.Dense(32, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(64, activation='relu', input_shape=(2,),
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(128, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(64, activation='relu', input_shape=(2,),
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(32, activation='relu',
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64),
        tf.keras.layers.Dense(1, activation=None,
                              kernel_initializer="glorot_uniform",
                              kernel_regularizer=regularizers.l2(regularization_strength),
                              dtype=tf.float64)
    ])
    return PINN



In [24]:
def checkpoint2_solution( x, y , t, regularization_strength):

  xmeas_train, xmeas_val, ymeas_train, ymeas_val, tmeas_train, tmeas_val = train_test_split(x, y, t, test_size=0.05)
  xmeas_train = tf.constant(xmeas_train.reshape(19, 1), dtype=tf.float64)
  ymeas_train = tf.constant(ymeas_train.reshape(19, 1), dtype=tf.float64)
  tmeas_train = tf.constant(tmeas_train.reshape(19, 1), dtype=tf.float64)
  xmeas_val = tf.constant(xmeas_val.reshape(1, 1), dtype=tf.float64)
  ymeas_val = tf.constant(ymeas_val.reshape(1, 1), dtype=tf.float64)
  tmeas_val = tf.constant(tmeas_val.reshape(1, 1), dtype=tf.float64)

  X, Y = np.meshgrid(np.linspace(-1.5,1.5,151), np.linspace(-1.5,1.5,151))

  coeff=tf.Variable([[0.5],[0.5],[0.5],[0.5],[0.5],[0.5],[0.5],[0.5],[0.5],[0.5]], trainable=True, dtype=tf.float64)

  tf_optimizer_PINN = tf.keras.optimizers.Adam(learning_rate=0.03,beta_1=0.99)
  tf_optimizer_weights = tf.keras.optimizers.Adam(learning_rate=0.01,beta_1=0.99)


  print()

  for iter in range(100):

    loss_value2,grads2 = grad2(xcl,ycl,xmeas_train, ymeas_train, tmeas_train,regularization_strength)

    tf_optimizer_PINN.apply_gradients(zip(grads2,build_pinn.trainable_variables))

    loss_value_val2, _= grad2( xcl, ycl, xmeas_val, ymeas_val, tmeas_val,regularization_strength)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value2)
      tf.print('loss_val_param =' , loss_value_val2)

      c1 = c1_min + delta_c1 * coeff[0]
      c2 = c2_min + delta_c2 * coeff[1]
      c3 = c3_min + delta_c3 * coeff[2]
      c4 = c4_min + delta_c4 * coeff[3]
      c5 = c5_min + delta_c5 * coeff[4]
      c6 = c6_min + delta_c6 * coeff[5]
      c7 = c7_min + delta_c7 * coeff[6]
      c8 = c8_min + delta_c8 * coeff[7]
      c9 = c9_min + delta_c9 * coeff[8]
      c10 = c10_min + delta_c10 * coeff[9]

      coeff_true = tf.expand_dims(tf.concat([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10], 0), 1)
      print(coeff_true.numpy())
      print()


  for iter in range(100):

    loss_value,grads,grads_coeff = grad(xcl,ycl,xmeas_train, ymeas_train, tmeas_train,coeff, regularization_strength)

    tf_optimizer_PINN.apply_gradients(zip(grads ,build_pinn.trainable_variables))
    tf_optimizer_weights.apply_gradients(zip([grads_coeff], [coeff]))

    loss_value_val, _, _ = grad( xcl, ycl, xmeas_val, ymeas_val, tmeas_val,coeff,regularization_strength)


    if ((iter+1) % 100 == 0):
      print('iter =  '+str(iter+1))
      tf.print('loss =' , loss_value)
      tf.print('loss_val_param =' , loss_value_val)

      c1 = c1_min + delta_c1 * coeff[0]
      c2 = c2_min + delta_c2 * coeff[1]
      c3 = c3_min + delta_c3 * coeff[2]
      c4 = c4_min + delta_c4 * coeff[3]
      c5 = c5_min + delta_c5 * coeff[4]
      c6 = c6_min + delta_c6 * coeff[5]
      c7 = c7_min + delta_c7 * coeff[6]
      c8 = c8_min + delta_c8 * coeff[7]
      c9 = c9_min + delta_c9 * coeff[8]
      c10 = c10_min + delta_c10 * coeff[9]

      coeff_true = tf.expand_dims(tf.concat([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10], 0), 1)
      print(coeff_true.numpy())
      print()



  return regularization_strength, loss_value_val

In [17]:
num_random_search = 10
regularization_strengths = np.logspace(-6, 0, num=num_random_search)
regularization_strengths

array([1.00000000e-06, 4.64158883e-06, 2.15443469e-05, 1.00000000e-04,
       4.64158883e-04, 2.15443469e-03, 1.00000000e-02, 4.64158883e-02,
       2.15443469e-01, 1.00000000e+00])

In [27]:
regularization_strength, loss_value_val= checkpoint2_solution(x_meas, y_meas, t_meas,regularization_strength=0.1)

TypeError: Binding inputs to tf.function failed due to `missing a required argument: 'regularization_strength'`. Received args: (<tf.Tensor: shape=(22801, 1), dtype=float64, numpy=
array([[ 0.22830158],
       [-0.11790101],
       [ 0.13515415],
       ...,
       [ 0.56502407],
       [ 0.29046929],
       [ 0.0540059 ]])>, <tf.Tensor: shape=(22801, 1), dtype=float64, numpy=
array([[-0.65131181],
       [ 0.97227213],
       [-0.47028016],
       ...,
       [ 0.4158394 ],
       [-1.18206263],
       [-0.03287141]])>, <tf.Tensor: shape=(19, 1), dtype=float64, numpy=
array([[ 0.9 ],
       [ 0.5 ],
       [-0.7 ],
       [ 1.04],
       [-0.7 ],
       [ 0.48],
       [ 0.34],
       [-0.28],
       [ 0.7 ],
       [-1.3 ],
       [-1.28],
       [-0.5 ],
       [-0.14],
       [ 1.3 ],
       [-1.5 ],
       [-1.48],
       [-0.1 ],
       [ 1.5 ],
       [-0.5 ]])>, <tf.Tensor: shape=(19, 1), dtype=float64, numpy=
array([[-0.94],
       [ 0.  ],
       [-0.12],
       [-1.08],
    

In [21]:
ind_disp = 64
x_meas = CP3data[ind_disp][0]
y_meas = CP3data[ind_disp][1]
t_meas = CP3data[ind_disp][2]
speed_field = CP3estimate[ind_disp][0]

num_random_search = 10
regularization_strengths = np.logspace(-6, 0, num=num_random_search)

best_val_loss = float('inf')
best_regularization_strength = None


for regularization_strength in regularization_strengths:
  regularization_strength, loss_value_val= checkpoint2_solution(x_meas, y_meas, t_meas,regularization_strength)
  if  loss_value_val < best_val_loss:
      best_val_loss =  loss_value_val
      best_regularization_strength = regularization_strength
  print(f'Best Regularization strength: {best_regularization_strength}, Best Validation Loss: { best_val_loss}')


TypeError: Binding inputs to tf.function failed due to `missing a required argument: 'regularization_strength'`. Received args: (<tf.Tensor: shape=(22801, 1), dtype=float64, numpy=
array([[ 0.22830158],
       [-0.11790101],
       [ 0.13515415],
       ...,
       [ 0.56502407],
       [ 0.29046929],
       [ 0.0540059 ]])>, <tf.Tensor: shape=(22801, 1), dtype=float64, numpy=
array([[-0.65131181],
       [ 0.97227213],
       [-0.47028016],
       ...,
       [ 0.4158394 ],
       [-1.18206263],
       [-0.03287141]])>, <tf.Tensor: shape=(19, 1), dtype=float64, numpy=
array([[-0.5 ],
       [-0.1 ],
       [ 1.5 ],
       [ 1.3 ],
       [-0.14],
       [-0.28],
       [ 0.34],
       [-0.5 ],
       [-1.28],
       [ 0.7 ],
       [ 0.48],
       [-0.7 ],
       [ 1.04],
       [-0.7 ],
       [-1.48],
       [-1.3 ],
       [ 0.5 ],
       [-0.32],
       [ 0.9 ]])>, <tf.Tensor: shape=(19, 1), dtype=float64, numpy=
array([[ 0.04],
       [ 0.48],
       [ 0.  ],
       [ 0.  ],
    